In [1]:
import pandas as pd
import numpy as np

In [2]:
users=['User1','User2','User3','User4','User5']
items=['Item1','Item2','Item3','Item4','Item5']
datasets=[
    [1,0,1,1,0],
    [1,0,0,1,1],
    [1,0,1,0,0],
    [0,1,0,1,1],
    [1,1,1,0,1],
]
df=pd.DataFrame(datasets,columns=items,index=users)
df

,Item1,Item2,Item3,Item4,Item5
User1,1,0,1,1,0
User2,1,0,0,1,1
User3,1,0,1,0,0
User4,0,1,0,1,1
User5,1,1,1,0,1


In [3]:
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import pairwise_distances


In [4]:
user_similar=1-pairwise_distances(df.values,metric='jaccard')#the calculation is for the intersection/union of rows with rows
user_similar

C:\Users\HUAWEI\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


array([[1.        , 0.5       , 0.66666667, 0.2       , 0.4       ],
       [0.5       , 1.        , 0.25      , 0.5       , 0.4       ],
       [0.66666667, 0.25      , 1.        , 0.        , 0.5       ],
       [0.2       , 0.5       , 0.        , 1.        , 0.4       ],
       [0.4       , 0.4       , 0.5       , 0.4       , 1.        ]])

In [5]:
user_similar=pd.DataFrame(user_similar,index=users,columns=users)

In [6]:
topN_users={}
#This loop is to find which user is similar to
for i in user_similar.index:
    #print(user_similar.loc[i])
    _df=user_similar.loc[i].drop([i])
    _df_sorted=_df.sort_values(ascending=False)
    top2= list(_df_sorted.index[:2])
    topN_users[i]=top2
print(topN_users)
print(df)
rs_results={}

{'User1': ['User3', 'User2'], 'User2': ['User1', 'User4'], 'User3': ['User1', 'User5'], 'User4': ['User2', 'User5'], 'User5': ['User3', 'User1']}
       Item1  Item2  Item3  Item4  Item5
User1      1      0      1      1      0
User2      1      0      0      1      1
User3      1      0      1      0      0
User4      0      1      0      1      1
User5      1      1      1      0      1


In [7]:
#This loop is to check which products are liked by similar users and then subtract the ones that the user already prefers
for user, sim_users in topN_users.items():
    rs_result=set()
    #print(rs_result)
    for sim_user in sim_users:
        rs_result=rs_result.union(set(df.loc[sim_user].replace(0,np.nan).dropna().index))#Taking the union here allows obtaining the names of items obtained in the first retrieval
        print(rs_result)
    rs_result -= set(df.loc[user].replace(0,np.nan).dropna().index)#Although User1 is similar to User2 and User3, and he also needs what User2 and User3 have, he must subtract the items he has already purchased
    rs_results[user]=rs_result#Assign to a dictionary and then refresh in a loop
rs_results

{'Item1', 'Item3'}
{'Item4', 'Item5', 'Item1', 'Item3'}
{'Item1', 'Item4', 'Item3'}
{'Item4', 'Item5', 'Item1', 'Item3', 'Item2'}
{'Item1', 'Item4', 'Item3'}
{'Item4', 'Item5', 'Item1', 'Item3', 'Item2'}
{'Item1', 'Item4', 'Item5'}
{'Item4', 'Item5', 'Item1', 'Item3', 'Item2'}
{'Item1', 'Item3'}
{'Item4', 'Item1', 'Item3'}


{'User1': {'Item5'},
 'User2': {'Item2', 'Item3'},
 'User3': {'Item2', 'Item4', 'Item5'},
 'User4': {'Item1', 'Item3'},
 'User5': {'Item4'}}